<a href="https://colab.research.google.com/github/Alisyed098/Capstone-Project---4-Online-Retail-Customer-Segmentation/blob/main/Capstone_Project_4_Online_Retail_Customer_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - **Online Retail Customer Segmentation**



##### **Project Type**    - Unsupervised
##### **Contribution**    - Individual
##### **Team Member 1 -** Syed Junaid Ali


# **Project Summary -**


This is an Unsupervised Learning project.I will have to build a model which is capable of segment retail customers on the basis of there Recency, Monetary and Frequency(RFM).



# **GitHub Link -**

Provide your GitHub Link here.

# **Problem Statement**


In this project, your task is to identify major customer segments on a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Import Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import datetime
import scipy
import scipy.stats as stat
import math
import sklearn

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
import plotly.express as px
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

## Libraries for Hierarchical clustering
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

### Dataset Loading

In [ ]:
# Load Dataset
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dframe = pd.read_excel('/content/drive/MyDrive/Online Retail.xlsx')

### Dataset First View

In [ ]:
# Dataset First Look
dframe.head()

In [ ]:
dframe.tail()

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
dframe.shape

### Dataset Information

In [ ]:
# Dataset Info
dframe.info()

#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
len(dframe[dframe.duplicated(keep = False)])

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count
dframe.isnull().sum()

In [ ]:
dframe['Country'].unique()

# Dataset Information

- InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
- StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
- Description: Product (item) name. Nominal.
- Quantity: The quantities of each product (item) per transaction. Numeric.
- InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated.
- UnitPrice: Unit price. Numeric, Product price per unit in sterling.
- CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
- Country: Country name. Nominal, the name of the country where each customer resides.


# **Exploratory Data Analysis**

In [ ]:
unique_df=pd.DataFrame()
unique_df['Features']=dframe.columns
unique=[]
for i in dframe.columns:
  unique.append(dframe[i].nunique())
unique_df['Uniques']=unique

f, ax = plt.subplots(1,1, figsize=(15,7))

splot = sns.barplot(x=unique_df['Features'], y=unique_df['Uniques'], alpha=0.8)
for p in splot.patches:
    splot.annotate(format(p.get_height(), '.0f'), (p.get_x() + p.get_width() / 2, p.get_height()), ha = 'center',
                   va = 'center', xytext = (0, 9), textcoords = 'offset points')
plt.title('Bar plot for number of unique values in each column',weight='bold', size=15)
plt.ylabel('#Unique values', size=12, weight='bold')
plt.xlabel('Features', size=12, weight='bold')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# distribution customer in 1 year period of time
plt.figure(figsize=(20,9))
customer_distribution = dframe.groupby('InvoiceDate').count()['CustomerID'].plot()

## Set title and x, y labels
plt.title('Distribution of customers Over period of 1 year', size=25)
plt.xlabel('Invoice date', size=15)
plt.ylabel('No. of Customers', size=15)
plt.show()

**Dropping Duplicates**

In [ ]:
# copying dataset
df = dframe.copy()

Before proceeding further dupicates values needs to be removed.

In [ ]:
# dropping all duplicate values
df.drop_duplicates(keep = "first", inplace = True)

In [ ]:
print(f"Shape of the dataset before removing duplicates{dframe.shape}")

print(f"Shape of the dataset after removing duplicates{df.shape}")

**Country count on the basis of Invoce no.**

In [ ]:
## Apply groupby function on country and Invoice No.
country_count = df.groupby('Country')['InvoiceNo'].count().reset_index()


In [ ]:
country_count.head()

In [ ]:
## From country_count variable finding top 10 countries
Top_ten_countries = country_count.sort_values('InvoiceNo', ascending=False, ignore_index=True).head(10)
Top_ten_countries

In [ ]:
## Adding country percentage column to top 10 countries data frame
Top_ten_countries['Country_percent']= Top_ten_countries['InvoiceNo']*100/df['InvoiceNo'].count()

In [ ]:
Top_ten_countries

In [ ]:
# Figure Size
plt.figure(figsize=(15,8))

## Define x and y axis
x=Top_ten_countries['Country']
y=Top_ten_countries['Country_percent']

# creating the bar plot
plt.bar(x,y,color ='c')

## Function to set label
def addlabels(x,y):
    for i in range(len(x)):
        plt.text(i,y[i],y[i],ha="center")

addlabels(x,round(y,2))

# set xlabel. ylabel and title
plt.xlabel("Country", size=20)
plt.ylabel("% Orders", size=20)
plt.title("% Orders from each Country", size=30)

## Showing the plot
plt.show()

**Checking for cancelled orders**

Lets look at the number of cancelled orders in the dataset.

In [ ]:
# String to be searched in start of string
search ="C"

# boolean series returned with False at place of NaN
bool_series = df["InvoiceNo"].str.startswith(search, na=False)

# displaying filtered dataframe
cancelled_orders = df[bool_series]

cancelled_orders.head()

In [ ]:
cancelled_orders.shape

In [ ]:
cancelled_orders_percentage = cancelled_orders['InvoiceNo'].count()*100/df['InvoiceNo'].count()
cancelled_orders_percentage

This shows that 9251 (approx 1.72%) were such orders which was cancelled.

In [ ]:
cancelled_orders_country = cancelled_orders.groupby('Country')['InvoiceNo'].count().reset_index()

In [ ]:
cancelled_orders_countries_top10 = cancelled_orders_country.sort_values('InvoiceNo',ascending=False, ignore_index=True).head(10)
cancelled_orders_countries_top10

In [ ]:
# Figure Size
plt.figure(figsize=(15,8))

## Define x and y axis
x=cancelled_orders_countries_top10['Country']
y=cancelled_orders_countries_top10['InvoiceNo']

# creating the bar plot
plt.bar(x,y,color = '#F8766D')

## Function to set label
def addlabels(x,y):
    for i in range(len(x)):
        plt.text(i,y[i],y[i],ha = "center")

addlabels(x,y)

# set xlabel. ylabel and title
plt.xlabel("Countries", size=20)
plt.ylabel("No. of Cancelled Orders", size=20)
plt.title("No. of Cancelled Orders from each Country", size=30)

## Showing the plot
plt.show()

In [ ]:
df = df[~df.InvoiceNo.str.contains("C", na=False)]

In [ ]:
# shape of dataset without cancelled orders
df.shape

**Customer's data**

In [ ]:
# defining new dataset for customer
cust_df = df.copy()

In [ ]:
# creating new column amount
cust_df['amount'] = cust_df.Quantity*cust_df.UnitPrice

In [ ]:
cust_df.head()

In [ ]:
# total sales per customer
Sales_sum = cust_df.groupby(["CustomerID"]).amount.sum().sort_values(ascending = False).iloc[:20].reset_index()
Sales_sum.head(10)

In [ ]:
sns.set(rc={'figure.figsize':(18,8)})
sns.barplot(x="CustomerID", y="amount", data=Sales_sum, order=Sales_sum['CustomerID'], palette = "mako")

**Top products**

In [ ]:
AmoutSum = cust_df.groupby(["Description"]).amount.sum().sort_values(ascending = False)
inv = cust_df[["Description", "InvoiceNo"]].groupby(["Description"]).InvoiceNo.unique().\
      agg(np.size).sort_values(ascending = False)

fig = plt.figure(figsize=(25, 7))
f1 = fig.add_subplot(121)
Top10 = list(AmoutSum[:10].index)
PercentSales =  np.round((AmoutSum[Top10].sum()/AmoutSum.sum()) * 100, 2)

g = AmoutSum[Top10].\
    plot(kind='bar', title='Top 10 Products in Sales Amount are having: {:3.2f}% of Amount'.\
                       format(PercentSales))

**Checking orders with negative quantities that were not cancelled**

In [ ]:
df_negative_qty = df[df['Quantity'] < 0]
df_negative_qty.head()

As we can see from the above, these cases are the ones where CustomerID values are NaNs. These cases were also removed from the data.

In [ ]:
# checking the shape
df_negative_qty.shape

So, there are 1336 rows having negative quantity and also orders were not canceled.

Also we can see that, these cases are the ones where CustomerID values are NaNs. These cases should be removed from the data.

In [ ]:
# defining dataset without negative quantities
df = df[df['Quantity'] > 0]

In [ ]:
# checking its shape...
df.shape

In [ ]:
df.isnull().sum()

Customer ID has 133322 Null Values. And Description has 592 Nan Values.Decided to drop the Null values to make the data more cleaned.

In [ ]:
# dropping null values
df.dropna(inplace=True)

In [ ]:
df.head()

In [ ]:
# checking shape after removing null values
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
# describing the dataset
df.iloc[:, :-1].describe().T.sort_values(by='std' , ascending = False)\
                     .style.background_gradient(cmap='Greens')\
                     .bar(subset=["max"], color='#F8766D')\
                     .bar(subset=["mean",], color='#00BFC4')

# **Feature Engineering &  Data Preprocessing** - **RFM model (Recency, Frequency, Monetary value)**

**Recency**

To create a Recency feature variable, i need to decide the reference date for analysis and i wil define the reference date as one day before the last transaction.

In [ ]:
reference_date = df.InvoiceDate.max()

#timedelta function returns to total number of seconds
reference_date = reference_date + datetime.timedelta(days=1)

print(df.InvoiceDate.max(), df.InvoiceDate.min())
reference_date

i will construct a reference variable as number of days before the reference date when a customer last made a purchase.

In [ ]:
df['days_since_last_purchase'] = reference_date- df.InvoiceDate
df['days_since_last_purchase_num'] = df['days_since_last_purchase'].astype('timedelta64[D]')
df['days_since_last_purchase_num'].head()

In [ ]:
customer_history_df = df.groupby('CustomerID')['days_since_last_purchase_num'].min().reset_index()
customer_history_df.rename(columns={'days_since_last_purchase_num':'Recency'}, inplace=True)
customer_history_df.head()

In [ ]:
customer_history_df.iloc[:, :].describe().T.sort_values(by='std' , ascending = False)\
                     .style.background_gradient(cmap='Greens')\
                     .bar(subset=["max"], color='#F8766D')\
                     .bar(subset=["mean",], color='#00BFC4')

**Frequency and Monetary value**

In [ ]:
# calculating monetary
df['Monetary'] = df['Quantity']*df['UnitPrice']

In [ ]:
customer_monetary_val = df[['CustomerID','Monetary']].groupby('CustomerID').sum().reset_index()
customer_monetary_val.head()

In [ ]:
customer_history_df=customer_history_df.merge(customer_monetary_val,how='outer')
customer_history_df.head()

In [ ]:
# calculating customer frequency
customer_freq = df[['CustomerID','Monetary']].groupby('CustomerID').count().reset_index()
customer_freq.rename(columns={'Monetary':'Frequency'},inplace=True)
customer_freq.head()

In [ ]:
customer_history_df=customer_history_df.merge(customer_freq,how='outer')
customer_history_df.head()

In [ ]:
customer_history_df.Monetary[customer_history_df.Monetary<=0].count()

In [ ]:
customer_history_df = customer_history_df[customer_history_df.Monetary != 0]

**Outlier Treatment**

In [ ]:
# Box plot of Recency
plt.rcParams['figure.figsize'] = [10,8]
sns.boxplot(data = customer_history_df['Recency'], orient="v", palette="Set2" ,whis=1.5,saturation=1, width=0.7)
plt.title("Outliers Variable Distribution", fontsize = 14, fontweight = 'bold')
plt.ylabel("Range", fontweight = 'bold')
plt.xlabel("Attribute - Recency", fontweight = 'bold')

In [ ]:
# Box plot of Monetary
plt.rcParams['figure.figsize'] = [10,8]
sns.boxplot(data = customer_history_df['Monetary'], orient="v", palette="Set2" ,whis=1.5,saturation=1, width=0.7)
plt.title("Outliers Variable Distribution", fontsize = 14, fontweight = 'bold')
plt.ylabel("Range", fontweight = 'bold')
plt.xlabel("Attribute - Monetary", fontweight = 'bold')

In [ ]:
# Box plot of Frequency
plt.rcParams['figure.figsize'] = [10,8]
sns.boxplot(data = customer_history_df['Frequency'], orient="v", palette="Set2" ,whis=1.5,saturation=1, width=0.7)
plt.title("Outliers Variable Distribution", fontsize = 14, fontweight = 'bold')
plt.ylabel("Range", fontweight = 'bold')
plt.xlabel("Attribute - Frequency", fontweight = 'bold')

In [ ]:
# removing (statistical) outliers
Q1 = customer_history_df.Monetary.quantile(0.05)
Q3 = customer_history_df.Monetary.quantile(0.95)
IQR = Q3 - Q1
customer_history_df = customer_history_df[(customer_history_df.Monetary >= Q1 - 1.5*IQR) & (customer_history_df.Monetary <= Q3 + 1.5*IQR)]

# outlier treatment for recency
Q1 = customer_history_df.Recency.quantile(0.05)
Q3 = customer_history_df.Recency.quantile(0.95)
IQR = Q3 - Q1
customer_history_df = customer_history_df[(customer_history_df.Recency >= Q1 - 1.5*IQR) & (customer_history_df.Recency <= Q3 + 1.5*IQR)]

# outlier treatment for frequency
Q1 = customer_history_df.Frequency.quantile(0.05)
Q3 = customer_history_df.Frequency.quantile(0.95)
IQR = Q3 - Q1
customer_history_df = customer_history_df[(customer_history_df.Frequency >= Q1 - 1.5*IQR) & (customer_history_df.Frequency <= Q3 + 1.5*IQR)]


In [ ]:
# Rescaling
rfm_df = customer_history_df[['Monetary', 'Frequency', 'Recency']]

# instantiate
scaler = StandardScaler()

# fit_transform
rfm_df_scaled = scaler.fit_transform(rfm_df)
rfm_df_scaled.shape

In [ ]:
rfm_df_scaled = pd.DataFrame(rfm_df_scaled)
rfm_df_scaled.columns = ['Monetary', 'Frequency', 'Recency']
rfm_df_scaled.head()

In [ ]:
# Importing Library for 3D plot
from mpl_toolkits.mplot3d import Axes3D

# set fig size
fig=plt.figure(figsize=(10,8))

# syntax for 3-D plotting
ax=fig.add_subplot(111,projection='3d')

# syntax for plotting
xs = rfm_df_scaled.Recency
ys = rfm_df_scaled.Frequency
zs = rfm_df_scaled.Monetary
ax.scatter(xs,ys,zs,s=5)

# set x, y & z label
ax.set_xlabel('Recency')
ax.set_ylabel('Frequency')
ax.set_zlabel('Monetary value')

# show the plot
plt.show()

**Model Training**

**Silhouette analysis on K-Means Clustering**

Silhouette analysis can be used to study the separation distance between the resulting clusters, as a strategy to quantifying the quality of clustering via graphical tool to plot a measure of how tightly grouped the samples in the clusters are. The silhouette plot displays a measure of how close each point in one cluster is to points in the neighboring clusters and thus provides a way to assess parameters like number of clusters visually.

Let's see below how our data perform for each K clusters groups (3, 5 and 7) in the silhouette score of each cluster, along with the center of each of the cluster discovered in the scatter plots, by amount_log vs recency_log and vs frequency_log.

Silhouette Analysis

silhouette score = (p-q)/max(p,q)

p is the mean distance to the points in the nearest cluster that the data point is not a part of

q is the mean intra-cluster distance to all the points in its own cluster.

The value of the silhouette score range lies between -1 to 1.

A score closer to 1 indicates that the data point is very similar to other data points in the cluster,

A score closer to -1 indicates that the data point is not similar to the data points in its cluster.

In [ ]:
# applying with 3 clusters
model = SilhouetteVisualizer(KMeans(3))
model.fit(rfm_df_scaled) # fitting the model

In [ ]:
# applying with 4 clusters
model = SilhouetteVisualizer(KMeans(4))
model.fit(rfm_df_scaled) # fitting

In [ ]:
# applying with 5 clusters
model = SilhouetteVisualizer(KMeans(5))
model.fit(rfm_df_scaled) # fitting

**Finding best value for k**

**The Elbow Method** - Using the elbow method to find the optimal number of clusters. The idea behind the elbow method is to identify the value of k where the distortion begins to increase most rapidly. If k increases, the distortion will decrease, because the samples will be closer to the centroids they are assigned to.

In [ ]:
# elbow-curve/SSD
ssd = []
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]
for num_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50)
    kmeans.fit(rfm_df_scaled)

    ssd.append(kmeans.inertia_)

# plot the SSDs for each n_clusters
# ssd
plt.plot(ssd)

In [ ]:
# silhouette analysis
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]

for num_clusters in range_n_clusters:

    # intialise kmeans
    kmeans = KMeans(n_clusters=num_clusters, max_iter=50)
    kmeans.fit(rfm_df_scaled)

    cluster_labels = kmeans.labels_

    # silhouette score
    silhouette_avg = silhouette_score(rfm_df_scaled, cluster_labels)
    print("For n_clusters={0}, the silhouette score is {1}".format(num_clusters, silhouette_avg))

In [ ]:
# final model with k=3
kmeans = KMeans(n_clusters=3, max_iter=50)
kmeans.fit(rfm_df_scaled)

In [ ]:
kmeans.labels_

In [ ]:
# assign the label
customer_history_df['cluster_id'] = kmeans.labels_
customer_history_df.head()

In [ ]:
# plot
sns.boxplot(x='cluster_id', y='Recency', data=customer_history_df)

In [ ]:
# plot
sns.boxplot(x='cluster_id', y='Monetary', data=customer_history_df)

In [ ]:
# plot
sns.boxplot(x='cluster_id', y='Frequency', data=customer_history_df)

**Hierarchical Clustering**

In [ ]:
rfm_df_scaled.head()

In [ ]:
customer_history_df.head()

In [ ]:
# single linkage
plt.figure(figsize=(15,10))
mergings = linkage(rfm_df_scaled, method="single", metric='euclidean')
dendrogram(mergings)
plt.show()

In [ ]:
# complete linkage
plt.figure(figsize=(18,10))
mergings = linkage(rfm_df_scaled, method="complete", metric='euclidean')
dendrogram(mergings)
plt.show()

In [ ]:
# 3 clusters
cluster_labels = cut_tree(mergings, n_clusters=3).reshape(-1, )
cluster_labels

In [ ]:
# assign cluster labels
customer_history_df['cluster_labels'] = cluster_labels
customer_history_df.head()

In [ ]:
# plots
sns.boxplot(x='cluster_labels', y='Recency', data=customer_history_df)

In [ ]:
# plots
sns.boxplot(x='cluster_labels', y='Monetary', data=customer_history_df)

In [ ]:
# plots
sns.boxplot(x='cluster_labels', y='Frequency', data=customer_history_df)

We can obeserved that cluster 2 have a highest sales frequency then to other clusters.

In [ ]:
### visualize the result
import plotly.express as px
fig = px.scatter_3d(customer_history_df, x='Recency', y='Frequency', z='Monetary',
              color='cluster_labels')
fig.show()

**Summary**

RFM analysis can segment customers into homogenous group quickly with set of minimum variables. Scoring system can be defined and ranged differently. We get a better result for clustering steps by applying scoring rather than using the raw calculated RFM values. Therefore, segmenting should be done by RFM scoring and further analysis on the spending behavior should be done on the raw values for the targeted cluster to expose more insight and characteristics. RFM analysis solely depends on purchasing behavior and histories, analysis can be further improved by exploring weighted composite scoring or including customer demographic information and product information. A good analysis can increase effectiveness and efficiency of marketing plans, hence increase profitability at minimum cost.

# **Conclusion**

To conclude, we saw how we can segment our customer depending on our business requirements. You can perform RFM for your entire customer base, or just a subset. For example, you may first segment customers based on a geographical area or other demographics, and then by RFM for historical, transaction-based behaviour segments. RFM analysis can help in answering many questions with respect to their customers and this can help companies to make marketing strategies for their customers, retaining their slipping customers and providing recommendations to their customer based on their interest. We used the K-means algorithm to segment our customer in various clusters having similar similarity. I think K-means did a pretty good job here.